In [19]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import os

In [20]:
ndcs = pd.read_csv("../Data/20240118_countries.csv")
ndcs.head(5)

,ISO,Country,NDC,Date,Climate.Watch.HTML.File
0,AFG,Afghanistan,1,11/23/16,AFG-first_ndc-EN.html
1,AGO,Angola,1,11/16/20,AGO-first_ndc-EN.html
2,AGO,Angola,2,5/31/21,AGO-revised_first_ndc-EN.html
3,ALB,Albania,1,9/21/16,ALB-first_ndc-EN.html
4,ALB,Albania,2,10/12/21,ALB-revised_first_ndc-EN.html


In [21]:
def extract_ndc(index):
    path_name = "../Data/20240117_ClimateWatch_AllData/NDC_text_HTML/ndc-master" + "/"

In [22]:
pd.read_csv("../Output/20240118_ndc_lines.csv")

,iso,country,ndc,date,html,elem,h1,h2,h3,h4,p,ol,ul,tab,hdr1,hdr2,hdr3,hdr4,line
0,AFG,Afghanistan,1,11/23/16,AFG-first_ndc-EN.html,10,2,1,0,0,1,0,0,0,Intended Nationally Determined Contribution,Submission to the United Nations Framework Con...,NaN,NaN,21 September 2015
1,AFG,Afghanistan,1,11/23/16,AFG-first_ndc-EN.html,11,2,1,0,0,2,0,0,0,Intended Nationally Determined Contribution,Submission to the United Nations Framework Con...,NaN,NaN,The Islamic Republic of Afghanistan hereby com...
2,AFG,Afghanistan,1,11/23/16,AFG-first_ndc-EN.html,13,2,1,0,0,0,0,0,1,Intended Nationally Determined Contribution,Submission to the United Nations Framework Con...,NaN,NaN,Executive Summary Base Year: 2005 Target Years...
3,AFG,Afghanistan,1,11/23/16,AFG-first_ndc-EN.html,42,2,1,0,0,0,0,1,0,Intended Nationally Determined Contribution,Submission to the United Nations Framework Con...,NaN,NaN,Adaptation: USD 10.785 billion\nMitigation: US...
4,AFG,Afghanistan,1,11/23/16,AFG-first_ndc-EN.html,45,2,1,0,0,3,0,0,0,Intended Nationally Determined Contribution,Submission to the United Nations Framework Con...,NaN,NaN,\nFigure 1. Greenhouse Gas Emissions for Afgha...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111147,ZWE,Zimbabwe,2,9/24/21,ZWE-revised_first_ndc-EN.html,1749,2,10,4,7,778,0,0,0,GOVERNMENT OF ZIMBABWE,"7 Information for Clarity, Transparency and Un...",4.6.4 Waste,Tracking climate finance in Zimbabwe,"11th Floor, Kaguvi Building,"
111148,ZWE,Zimbabwe,2,9/24/21,ZWE-revised_first_ndc-EN.html,1750,2,10,4,7,779,0,0,0,GOVERNMENT OF ZIMBABWE,"7 Information for Clarity, Transparency and Un...",4.6.4 Waste,Tracking climate finance in Zimbabwe,"Cnr S.V Muzenda Street/ Central Avenue, Harare..."
111149,ZWE,Zimbabwe,2,9/24/21,ZWE-revised_first_ndc-EN.html,1751,2,10,4,7,780,0,0,0,GOVERNMENT OF ZIMBABWE,"7 Information for Clarity, Transparency and Un...",4.6.4 Waste,Tracking climate finance in Zimbabwe,Tel: +263 (242) 701681/3
111150,ZWE,Zimbabwe,2,9/24/21,ZWE-revised_first_ndc-EN.html,1752,2,10,4,7,781,0,0,0,GOVERNMENT OF ZIMBABWE,"7 Information for Clarity, Transparency and Un...",4.6.4 Waste,Tracking climate finance in Zimbabwe,Email: climatechange@environment.gov.zw Websit...


In [38]:
def extract_ndc(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        soup = BeautifulSoup(file, 'html.parser')
        
    # Extracting elements
    elements = soup.find_all()
    h1 = soup.find_all('h1')
    h2 = soup.find_all('h2')
    h3 = soup.find_all('h3')
    h4 = soup.find_all('h4')
    p = soup.find_all('p')
    ul = soup.find_all('ul')
    ol = soup.find_all('ol')
    tab = soup.find_all('table')
    
    # Creating a DataFrame to store the information
    struct = pd.DataFrame({
        'elem': range(len(elements)),
        'h1': [0]*len(elements),
        'h2': [0]*len(elements),
        'h3': [0]*len(elements),
        'h4': [0]*len(elements),
        'p': [0]*len(elements),
        'ol': [0]*len(elements),
        'ul': [0]*len(elements),
        'tab': [0]*len(elements)
    })
    
    # Marking the presence of tags
    struct.loc[struct['elem'].isin([i for i, e in enumerate(elements) if e in h1]), 'h1'] = 1
    struct.loc[struct['elem'].isin([i for i, e in enumerate(elements) if e in h2]), 'h2'] = 1
    struct.loc[struct['elem'].isin([i for i, e in enumerate(elements) if e in h3]), 'h3'] = 1
    struct.loc[struct['elem'].isin([i for i, e in enumerate(elements) if e in h4]), 'h4'] = 1
    struct.loc[struct['elem'].isin([i for i, e in enumerate(elements) if e in p]), 'p'] = 1
    struct.loc[struct['elem'].isin([i for i, e in enumerate(elements) if e in ul]), 'ul'] = 1
    struct.loc[struct['elem'].isin([i for i, e in enumerate(elements) if e in ol]), 'ol'] = 1
    struct.loc[struct['elem'].isin([i for i, e in enumerate(elements) if e in tab]), 'tab'] = 1
    
    # Cumulative sums for tags
    struct['h1'] = struct['h1'].cumsum()
    struct['h2'] = struct['h2'].cumsum()
    struct['h3'] = struct['h3'].cumsum()
    struct['h4'] = struct['h4'].cumsum()
    struct['p'] = struct['p'].cumsum() * struct['p']
    struct['ol'] = struct['ol'].cumsum() * struct['ol']
    struct['ul'] = struct['ul'].cumsum() * struct['ul']
    struct['tab'] = struct['tab'].cumsum() * struct['tab']
    
    # Filtering rows with at least one of the specified tags
    struct2 = struct[struct[['p', 'ol', 'ul', 'tab']].sum(axis=1) > 0]

    # print(struct)
    
    # Extracting text from elements
    # print([e for e in elements])
    struct2
    struct2['line'] = [e.get_text() for e in elements]
    
    # Adding header texts
    struct2['hdr1'] = [e.get_text() for e in h1]
    struct2['hdr2'] = [e.get_text() for e in h2]
    struct2['hdr3'] = [e.get_text() for e in h3]
    struct2['hdr4'] = [e.get_text() for e in h4]
    # if e in struct2['elem'].values
    # Adding additional information from the ndcs DataFrame
    struct2['iso'] = ndcs['ISO'].iloc[0]
    struct2['country'] = ndcs['Country'].iloc[0]
    struct2['ndc'] = ndcs['NDC'].iloc[0]
    struct2['date'] = ndcs['Date'].iloc[0]
    struct2['html'] = file_path
    
    return struct2

In [39]:
ndc_lines = extract_ndc("../Data/20240117_ClimateWatch_AllData/NDC_text_HTML/ndc-master/AFG-first_ndc-EN.html")
ndc_lines
# # Filtering out empty lines
# ndc_lines = ndc_lines[ndc_lines['line'] != '']

ValueError: Length of values (438) does not match length of index (59)